In [1]:
import pandas as pd

from transformers import pipeline
from datasets import load_dataset
from evaluate import load

In [2]:
dataset = load_dataset(
    "csv",
    data_files={
        "train": "Datasets/TrainSummary.csv",
        "validation": "Datasets/ValidationSummary.csv",
        "test": "Datasets/TestSummary.csv"
    },
)
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'summary'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'summary'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'summary'],
        num_rows: 200
    })
})

In [3]:
rouge = load("rouge")
bleu = load("sacrebleu")

In [4]:
def calculate_scores(dataset, model_path):
    summarizer = pipeline("summarization", model=model_path, device=0)

    references = dataset["summary"]
    predictions = []

    summary = summarizer(
        dataset["text"], max_length=100, truncation=True
    )
    for dict in summary:
        predictions.append(dict["summary_text"])

    rouge_scores = rouge.compute(references=references, predictions=predictions)
    bleu_scores = bleu.compute(references=references, predictions=predictions)

    return rouge_scores, bleu_scores

In [5]:
rouge_scores1, bleu_scores1 = calculate_scores(dataset['test'], "BART/checkpoint-1")

Your max_length is set to 100, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
c:\Users\Mrinal Devnath\anaconda3\envs\transformer\lib\site-packages\transformers\models\bart\modeling_bart.py:590: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Your max_length is set to 100, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your max_length is set to 100, but your input_length is only 30. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might cons

In [6]:
rouge_scores2, bleu_scores2 = calculate_scores(dataset['test'], "PEGASUS/checkpoint-1")

Your max_length is set to 100, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)
Your max_length is set to 100, but your input_length is only 36. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your max_length is set to 100, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)
Your max_length is set to 100, but your input_length is only 21. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Your

In [7]:
rouge_scores3, bleu_scores3 = calculate_scores(dataset['test'], "T5/checkpoint-1")

Your max_length is set to 100, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your max_length is set to 100, but your input_length is only 43. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)
Your max_length is set to 100, but your input_length is only 34. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)
Your max_length is set to 100, but your input_length is only 25. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Your

In [8]:
rs = [rouge_scores1, rouge_scores2, rouge_scores3]
rscolumn_names = list(rouge_scores1.keys())
rouge_scores = pd.DataFrame(rs, columns=rscolumn_names, index=["BART", "PEGASUS", "T5"])

print("ROUGE scores for the finetuned models: ")
rouge_scores

ROUGE scores for the finetuned models: 


,rouge1,rouge2,rougeL,rougeLsum
BART,0.410772,0.180822,0.342321,0.388334
PEGASUS,0.404608,0.183510,0.341101,0.372523
T5,0.291501,0.118733,0.233956,0.260050


In [9]:
bs = [bleu_scores1, bleu_scores2, bleu_scores3]
bscolumn_names = list(bleu_scores1.keys())
bleu_scores = pd.DataFrame(bs, columns=bscolumn_names, index=["BART", "PEGASUS", "T5"])

print("BLEU scores for the finetuned models: ")
bleu_scores

BLEU scores for the finetuned models: 


,score,counts,totals,precisions,bp,sys_len,ref_len
BART,13.241628,"[2529, 1016, 559, 356]","[6690, 6490, 6290, 6090]","[37.80269058295964, 15.654853620955317, 8.8871...",1.000000,6690,6165
PEGASUS,12.430868,"[2131, 860, 480, 324]","[5364, 5164, 4967, 4774]","[39.727815063385535, 16.65375677769171, 9.6637...",0.861286,5364,6165
T5,10.222876,"[1986, 795, 420, 270]","[6667, 6467, 6267, 6067]","[29.788510574471278, 12.293180763878151, 6.701...",1.000000,6667,6165
